In [ ]:
!pip install minio

In [ ]:
import os
from minio import Minio
import requests

In [ ]:
client = Minio("minio-service.kubeflow.svc.cluster.local:9000",
    access_key="minio",
    secret_key="minio123",
    secure=False,           
)

In [ ]:
type(client)

In [ ]:
bucket_name = "newtiledb"

In [ ]:
def upload_files(bucket_name, file_location, client):
    found = False  # Initialize 'found' before the try block
    print("Current working directory:", os.getcwd())
    print("Listing directories in the current working directory:", os.listdir("."))
    print(f"Checking existence of {file_location}: ", os.path.exists(file_location))

    try:
        found = client.bucket_exists(bucket_name)
    except Exception as e:
        print("error trying to search for MinIO Bucket:", e)
        return  # Return early since we cannot proceed without knowing if the bucket exists

    if not found:
        try:
            client.make_bucket(bucket_name)
            print("Created bucket", bucket_name)
        except Exception as e:
            print("Failed to create bucket:", e)
            return  # Return early since we cannot proceed if the bucket cannot be created
    else:
        print("Bucket", bucket_name, "exists, we won't attempt to create one")
        
    # Ensure the directory exists
    if not os.path.isdir(file_location):
        print(f"The directory {file_location} does not exist.")
        return

    # Iterate through all files in the directory
    for file_name in os.listdir(file_location):
        # Construct the full file path
        source_file = os.path.join(file_location, file_name)
        # Check if it's a file and not a directory
        if os.path.isfile(source_file):
            try:
                # Upload the file
                client.fput_object(bucket_name, file_name, source_file)
                print(f"Successfully uploaded {file_name} to bucket {bucket_name}.")
            except Exception as e:
                print(f"Failed to upload {file_name}: {e}")


In [ ]:
data = {
  "instances": [{
    "input": "When was Kubeflow Open-Sourced?",
    "num_docs": 6  # number of documents to retrieve
  }]
}

URL = "http://vectorstore-predictor.christensenc3526.svc.cluster.local/v1/models/vectorstore:predict"  # Adjust path as necessary

response = requests.post(URL, json=data, verify=False)  # 'verify=False' for self-signed certs
print(response)
#print(response.json())
print(response.text)

In [ ]:
!curl -X GET http://llm-predictor.christensenc3526.svc.cluster.local/v1/models

In [ ]:
!curl -X GET http://llm.christensenc3526.svc.cluster.local/v1/models

# run this command from your terminal
```
curl -X POST http://llm-predictor.christensenc3526.svc.cluster.local/v1/chat/completions \
  -H "Content-Type: application/json" \
  -d '{
    "model": "meta/llama-2-7b-chat",
    "messages": [{"role": "user", "content": "What is KServe?"}],
    "temperature": 0.5,
    "top_p": 1,
    "max_tokens": 256,
    "stream": false
  }'
``` 

In [ ]:
data = {
  "instances": [{
    "input": "Is Kubeflow for ML?",
    "num_docs": 6  # number of documents to retrieve
  }]
}

URL = "http://llm.christensenc3526.svc.cluster.local/v1/models/llm:predict"  # Adjust path as necessary

response = requests.post(URL, json=data, verify=False)  # 'verify=False' for self-signed certs
print(response)
#print(response.json())
print(response.text)

In [ ]:
this works 

In [ ]:
curl -X POST http://llm-transformer.christensenc3526.svc.cluster.local/v1/models/llm:predict \
  -H "Content-Type: application/json" \
  -d '{
    "instances": [{"input": "What is KServe?", "num_docs": 4}]
  }'


In [ ]:
# Serving a Hugging Face Model

[docs](https://kserve.github.io/website/latest/modelserving/v1beta1/llm/torchserve/accelerate/#package-the-model)

In [ ]:
!pip install transformers

In [ ]:
!pip install torch 

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "meta-llama/Llama-2-7b-chat-hf"
save_path = "./llama-2-7b-chat"

# Load and save model
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)


OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Llama-2-7b-chat-hf.
401 Client Error. (Request ID: Root=1-679c3ba9-0931e4e01c81ed460a001bfc;b9928a49-a6e7-4c83-b307-36043390b122)

Cannot access gated repo for url https://huggingface.co/meta-llama/Llama-2-7b-chat-hf/resolve/main/config.json.
Access to model meta-llama/Llama-2-7b-chat-hf is restricted. You must have access to it and be authenticated to access it. Please log in.